In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Aug 25 09:37:54 2018

@author: antoine
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
# import statsmodels.regression.linear_model as sm
import statsmodels.api as sm

# importer le dataset
dataset = pd.read_csv("50_Startups.csv")
X = dataset.loc[:,"R&D Spend":"State"].values
# X = dataset.drop(columns="Profit").values
Y = dataset.loc[:,"Profit"].values

# Encodage des variables de catégories
labelencoder_X = LabelEncoder()
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])

onehotencoder_X = OneHotEncoder(categorical_features =[3])
X = onehotencoder_X.fit_transform(X).toarray()

# Pièges des variables factices
X = X[:, 1:]

# Diviser notre dataset en training set et test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Feature Scaling <=> Normalisation 
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Fittage du modèle de régression
regressor = LinearRegression(fit_intercept=True)
regressor.fit(X_train, y_train)

# Prédiction des résultats
y_pred = regressor.predict(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [5]:
"""
# Stats Models For backward elimination
"""

mse = mean_squared_error(y_pred, y_test)
print(np.sqrt(mse))

X_train1 = sm.add_constant(X_train)
regressor_OLS = sm.OLS(endog = y_train, exog = X_train1).fit()

X_test1 = sm.add_constant(X_test)
y_pred_ols = regressor_OLS.predict(X_test1)

print(regressor_OLS.summary())

X_opt = X_train1[:, [0,1,3,4,5]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
print(regressor_OLS.summary())

X_opt = X_train1[:, [0,3,4,5]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
print(regressor_OLS.summary())

X_opt = X_train1[:, [0,3,5]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
print(regressor_OLS.summary())

X_opt = X_train1[:, [0,3]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
print(regressor_OLS.summary())

regressor.fit(X_train[:, [2]], y_train)
y_pred_elim = regressor.predict(X_test[:, [2]])
mse_elim = mean_squared_error(y_pred_elim, y_test)
print(np.sqrt(mse_elim))

9137.99015279494
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     129.7
Date:                Sat, 09 Feb 2019   Prob (F-statistic):           3.91e-21
Time:                        12:06:53   Log-Likelihood:                -421.10
No. Observations:                  40   AIC:                             854.2
Df Residuals:                      34   BIC:                             864.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.094e+05   1548.918   

In [ ]:
# test